In [45]:
import argparse
import time
import matplotlib.pyplot as plt
import numpy as np
import optparse
from dotenv import load_dotenv
import random
import soundfile as sf
import sounddevice as sd

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowPresets
from brainflow import ml_model
from brainflow.ml_model import *

In [52]:
def get_random_clip_file(genre: str) -> str:
    i = random.randint(0, 9)
    return os.path.join("genre_clip_data", genre, f'{genre}.0000{i}.wav')

def play_random_clip_file(genre: str) -> None:
    data, fs = sf.read(get_random_clip_file(genre), dtype='float32')
    sd.play(data, fs)

def get_preds_for_genre(genre: str, t_steps_per_pred: int = 50, recording_time: float = 5.0):
    # prepare device
    load_dotenv()
    BoardShim.enable_dev_board_logger()

    board_id = 0
    serial_port = os.getenv("SERIAL_PORT")

    params = BrainFlowInputParams()
    params.serial_port = serial_port
    board = BoardShim(board_id, params)
    board.prepare_session()

    play_random_clip_file(genre)
    board.start_stream()
    time.sleep(recording_time)
    data = board.get_board_data()
    board.stop_stream()
    board.release_session()

    model = MLModel(BrainFlowModelParams(1, 0))
    num_preds = data.shape[1] // t_steps_per_pred
    preds = []
    for i in range(num_preds):
        model.prepare()
        prediction = model.predict(data[:, i * t_steps_per_pred : (i+1) * t_steps_per_pred])
        model.release()
        preds.append(prediction)
    
    return preds

In [55]:
np.mean(get_preds_for_genre("classical"))

0.0

In [34]:
# prepare device
load_dotenv()
BoardShim.enable_dev_board_logger()

board_id = 0
serial_port = os.getenv("SERIAL_PORT")

params = BrainFlowInputParams()
params.serial_port = serial_port
board = BoardShim(board_id, params)
board.prepare_session()
board.get_sampling_rate(board_id)   # per second, presumably

250

In [35]:
board.start_stream ()
time.sleep(2)
# data = board.get_current_board_data (256) # get latest 256 packages or less, doesnt remove them from internal buffer
data = board.get_board_data()  # get all data and remove it from internal buffer
board.stop_stream()
board.release_session()

In [47]:
model = MLModel(BrainFlowModelParams(0, 0))


In [48]:
t_steps_per_pred = 50
num_preds = data.shape[1] // t_steps_per_pred
for i in range(num_preds):
    model.prepare()
    prediction = model.predict(data[:, i * t_steps_per_pred : (i+1) * t_steps_per_pred])
    model.release()
    print(i, prediction)

0 [1.]
1 [1.]
2 [1.]
3 [1.]
4 [1.]
5 [1.]
6 [1.]
7 [1.]
